# Curating a genome scale model

This notebook has been tested on [jprime.lbl.gov](jprime.lbl.gov) with the biodesign_3.7 kernel.

It starts with the model that gets output by the annotation_gr.ipynb notebook.

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from IPython.display import IFrame
import numpy as np
import pandas as pd
import json
import urllib
import cobra
import cplex
import os
import requests
import collections
import itertools

# Getting and preparing the genome-scale model

## Load *R.opacus* NCBI model generated by CarveMe

In [2]:
model = cobra.io.read_sbml_model("GSMs/Ropacus_some_reaction_deletions.xml")
model

Name,ropacus_curated_with_some_deleted_reactions
Memory address,0x07fed972eefd0
Number of metabolites,1581
Number of reactions,2375
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


## Starting MEMOTE Output
Need to update

In [3]:
IFrame('memotes/ropacus_carveme_grampos.htm', 1500, 800)

# Look into Various methods to play with the GSM

#### follow simulating FBA documentation page 

In [11]:
model.objective = 'Growth'

In [12]:
solution = model.optimize()
print(solution)

<Solution 0.000 at 0x7fee49ebfdd0>


In [13]:
solution.objective_value

0.0

In [15]:
%%time
model.optimize().objective_value

CPU times: user 28 ms, sys: 0 ns, total: 28 ms
Wall time: 27.6 ms


0.0

In [16]:
%%time
model.slim_optimize()

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1.04 ms


0.0

In [17]:
model.summary()

In [18]:
model.metabolites.nadh_c.summary()

In [19]:

model.metabolites.atp_c.summary()

In [20]:
biomass_rxn = model.reactions.get_by_id("Growth")

In [21]:
from cobra.util.solver import linear_reaction_coefficients
linear_reaction_coefficients(model)

{<Reaction Growth at 0x7fee496f96d0>: 1.0}

In [22]:
# change the objective to ATPM
model.objective = "ATPM"

# The upper bound should be 1000, so that we get
# the actual optimal value
model.reactions.get_by_id("ATPM").upper_bound = 1000.
linear_reaction_coefficients(model)

{<Reaction ATPM at 0x7fee496f9910>: 1.0}

In [23]:
model.optimize().objective_value

65.0

In [24]:
model.summary()

In [25]:
from cobra.flux_analysis import flux_variability_analysis

In [26]:
flux_variability_analysis(model, model.reactions[:10])

,minimum,maximum
12DGR120tipp,0.0,0.0
12DGR140tipp,0.0,0.0
12DGR141tipp,0.0,0.0
12DGR161tipp,0.0,0.0
12DGR180tipp,0.0,0.0
12DGR181tipp,0.0,0.0
12PPDStpp,0.0,0.0
12PPDt,0.0,0.0
14GLUCANabcpp,0.0,0.0
14GLUCANtexi,0.0,0.0


In [27]:
cobra.flux_analysis.flux_variability_analysis(
    model, model.reactions[:10], fraction_of_optimum=0.9)

,minimum,maximum
12DGR120tipp,0.0,2.166667
12DGR140tipp,0.0,2.166667
12DGR141tipp,0.0,2.166667
12DGR161tipp,0.0,2.166667
12DGR180tipp,0.0,2.166667
12DGR181tipp,0.0,2.166667
12PPDStpp,0.0,0.000000
12PPDt,0.0,0.000000
14GLUCANabcpp,0.0,0.000000
14GLUCANtexi,0.0,0.000000


In [31]:
loop_reactions = [model.reactions[800], model.reactions[1500]]
flux_variability_analysis(model, reaction_list=loop_reactions, loopless=False)

,minimum,maximum
FAS140,0.0,0.0
NDPK3,0.0,0.0


In [29]:
model.reactions.get_by_id('FRD7')

KeyError: 'FRD7'

In [32]:
flux_variability_analysis(model, reaction_list=loop_reactions, loopless=True)

,minimum,maximum
FAS140,0.0,0.0
NDPK3,0.0,0.0


In [33]:
model.optimize()
model.summary(fva=0.95)

In [34]:
model.metabolites.pyr_c.summary(fva=0.95)

In [35]:

model.objective = 'Growth'
fba_solution = model.optimize()
pfba_solution = cobra.flux_analysis.pfba(model)

In [37]:
abs(fba_solution.fluxes["Growth"] - pfba_solution.fluxes[
    "Growth"])

0.0

In [38]:
fba_solution.fluxes["Growth"]

0.0

# Simulating Deletions

In [39]:
import pandas
from time import time

import cobra.test
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

In [ ]:
print('complete model: ', cobra_model.optimize())
with cobra_model:
    cobra_model.reactions.PFK.knock_out()
    print('pfk knocked out: ', cobra_model.optimize())

# Growth Media

In [41]:
model.medium

{'EX_glc__D_e': 10.0,
 'EX_h2o_e': 10.0,
 'EX_h_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_pi_e': 10.0,
 'EX_nh4_e': 10.0,
 'EX_fe3_e': 10.0,
 'EX_k_e': 10.0,
 'EX_ca2_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_mn2_e': 10.0,
 'EX_cobalt2_e': 10.0,
 'EX_zn2_e': 10.0,
 'EX_cu2_e': 10.0,
 'EX_o2_e': 10.0,
 'EX_fe2_e': 10.0,
 'EX_mobd_e': 10.0,
 'EX_so4_e': 10.0}

In [45]:
for x in model.medium:
    print(type(model.medium[x]))

<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>


This is a complicated way to set medium composition <br>
The reason is that model.medium is a copy of the medium. Changing it won't effect the medium. You have to assign a full dictionary to the medium to change it

In [48]:
medium = model.medium
medium['EX_o2_e'] = 0
model.medium = medium

model.medium

{'EX_glc__D_e': 10.0,
 'EX_h2o_e': 10.0,
 'EX_h_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_pi_e': 10.0,
 'EX_nh4_e': 10.0,
 'EX_fe3_e': 10.0,
 'EX_k_e': 10.0,
 'EX_ca2_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_mn2_e': 10.0,
 'EX_cobalt2_e': 10.0,
 'EX_zn2_e': 10.0,
 'EX_cu2_e': 10.0,
 'EX_fe2_e': 10.0,
 'EX_mobd_e': 10.0,
 'EX_so4_e': 10.0}

In [49]:
model.slim_optimize()

0.0

In [50]:
medium = model.medium
medium['EX_o2_e'] = 10
model.medium = medium

model.medium

{'EX_glc__D_e': 10.0,
 'EX_h2o_e': 10.0,
 'EX_h_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_pi_e': 10.0,
 'EX_nh4_e': 10.0,
 'EX_fe3_e': 10.0,
 'EX_k_e': 10.0,
 'EX_ca2_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_mn2_e': 10.0,
 'EX_cobalt2_e': 10.0,
 'EX_zn2_e': 10.0,
 'EX_cu2_e': 10.0,
 'EX_o2_e': 10,
 'EX_fe2_e': 10.0,
 'EX_mobd_e': 10.0,
 'EX_so4_e': 10.0}

In [51]:
from cobra.medium import minimal_medium

max_growth = model.slim_optimize()
minimal_medium(model, max_growth)

Series([], dtype: float64)

In [52]:
model.medium

{'EX_glc__D_e': 10.0,
 'EX_h2o_e': 10.0,
 'EX_h_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_pi_e': 10.0,
 'EX_nh4_e': 10.0,
 'EX_fe3_e': 10.0,
 'EX_k_e': 10.0,
 'EX_ca2_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_mn2_e': 10.0,
 'EX_cobalt2_e': 10.0,
 'EX_zn2_e': 10.0,
 'EX_cu2_e': 10.0,
 'EX_o2_e': 10,
 'EX_fe2_e': 10.0,
 'EX_mobd_e': 10.0,
 'EX_so4_e': 10.0}

In [53]:
minimal_medium(model, 0.1, minimize_components=True)

Minimization of medium was infeasible.


In [54]:
minimal_medium(model, 0.8, minimize_components=8, open_exchanges=True)

Minimization of medium was infeasible.


In [56]:
for r in model.exchanges:
    print(r)

EX_12ppd__S_e: 12ppd__S_e --> 
EX_glc__D_e: glc__D_e <=> 
EX_h2o_e: h2o_e <=> 
EX_14glucan_e: 14glucan_e --> 
EX_h_e: h_e <=> 
EX_hco3_e: hco3_e --> 
EX_2hxmp_e: 2hxmp_e --> 
EX_2pglyc_e: 2pglyc_e --> 
EX_34dhcinm_e: 34dhcinm_e --> 
EX_leu__L_e: leu__L_e --> 
EX_3hoxpac_e: 3hoxpac_e --> 
EX_3mb_e: 3mb_e --> 
EX_4hbz_e: 4hbz_e --> 
EX_ala__L_e: ala__L_e --> 
EX_cl_e: cl_e <=> 
EX_LalaDgluMdapDala_e: LalaDgluMdapDala_e --> 
EX_5mtr_e: 5mtr_e --> 
EX_arab__L_e: arab__L_e --> 
EX_4abut_e: 4abut_e --> 
EX_acac_e: acac_e --> 
EX_acald_e: acald_e --> 
EX_acgam_e: acgam_e --> 
EX_ac_e: ac_e --> 
EX_chol_e: chol_e --> 
EX_acmana_e: acmana_e --> 
EX_acnam_e: acnam_e --> 
EX_pi_e: pi_e <=> 
EX_acser_e: acser_e --> 
EX_actn__R_e: actn__R_e --> 
EX_acysbmn_e: acysbmn_e --> 
EX_adn_e: adn_e --> 
EX_ins_e: ins_e --> 
EX_nh4_e: nh4_e <=> 
EX_ad_e: ad_e --> 
EX_akg_e: akg_e --> 
EX_mal__L_e: mal__L_e --> 
EX_asn__L_e: asn__L_e --> 
EX_ala__D_e: ala__D_e --> 
EX_gln__L_e: gln__L_e --> 
EX_gly_e: gly_e -

In [57]:
for r in model.demands:
    print(r)

ATPM: atp_c + h2o_c --> adp_c + h_c + pi_c


In [65]:
for r in model.sinks:
    print(r)
    for m in r.metabolites:
        print(m.name)
    print()

sink_2ohph_c: 2ohph_c --> 
2-Octaprenyl-6-hydroxyphenol

sink_4hba_c: 4hba_c --> 
4-Hydroxy-benzyl alcohol

sink_5drib_c: 5drib_c --> 
5'-deoxyribose

sink_bmocogdp_c: bmocogdp_c --> 
Bis-molybdopterin guanine dinucleotide

sink_lipopb_c: lipopb_c --> 
Lipoate (protein bound)

sink_sheme_c: sheme_c --> 
Siroheme



What is the difference between a sink and an exchange?

Exchange reactions - are reactions that move metabolites across in silico compartments. These in silico compartments are representive of intra- and inter- cellular membranes.<br><br>
Sink reactions - The metabolites, produced in reactions that are outside of an ambit of the system or in unknown reactions, are supplied to the network with reversible sink reactions.<br><br>
Demand reactions - Irreversible reactions added to the model to consume metabolites that are deposited in the system.

In [67]:
for r in model.boundary:
    print(r)

EX_12ppd__S_e: 12ppd__S_e --> 
EX_glc__D_e: glc__D_e <=> 
EX_h2o_e: h2o_e <=> 
EX_14glucan_e: 14glucan_e --> 
EX_h_e: h_e <=> 
EX_hco3_e: hco3_e --> 
EX_2hxmp_e: 2hxmp_e --> 
EX_2pglyc_e: 2pglyc_e --> 
EX_34dhcinm_e: 34dhcinm_e --> 
EX_leu__L_e: leu__L_e --> 
EX_3hoxpac_e: 3hoxpac_e --> 
EX_3mb_e: 3mb_e --> 
EX_4hbz_e: 4hbz_e --> 
EX_ala__L_e: ala__L_e --> 
EX_cl_e: cl_e <=> 
EX_LalaDgluMdapDala_e: LalaDgluMdapDala_e --> 
EX_5mtr_e: 5mtr_e --> 
EX_arab__L_e: arab__L_e --> 
EX_4abut_e: 4abut_e --> 
EX_acac_e: acac_e --> 
EX_acald_e: acald_e --> 
EX_acgam_e: acgam_e --> 
EX_ac_e: ac_e --> 
EX_chol_e: chol_e --> 
EX_acmana_e: acmana_e --> 
EX_acnam_e: acnam_e --> 
EX_pi_e: pi_e <=> 
EX_acser_e: acser_e --> 
EX_actn__R_e: actn__R_e --> 
EX_acysbmn_e: acysbmn_e --> 
EX_adn_e: adn_e --> 
EX_ins_e: ins_e --> 
EX_nh4_e: nh4_e <=> 
EX_ad_e: ad_e --> 
EX_akg_e: akg_e --> 
EX_mal__L_e: mal__L_e --> 
EX_asn__L_e: asn__L_e --> 
EX_ala__D_e: ala__D_e --> 
EX_gln__L_e: gln__L_e --> 
EX_gly_e: gly_e -